In [ ]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install -U tf2onnx
!pip install onnxruntime
!pip install gdown==4.6.0

convert

In [ ]:
# Download public test
!gdown https://drive.google.com/uc?id=<dataset_id>&export=download
# Unzip dataset
!unzip -qq cropped_faces_private_test.zip
# !rm public_test_cropped_faces.zip

Downloading...
From: https://drive.google.com/uc?id=<datase_id>
To: /content/cropped_faces_private_test.zip
100% 323M/323M [00:01<00:00, 224MB/s]


In [ ]:
!pip install keras_cv
!pip install deepface

import numpy as np
import glob
from keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix
import time

import os
import gdown
import tensorflow as tf
from deepface.commons import functions
from deepface.commons.logger import Logger
from keras.layers import Dense, Activation, Flatten, Dropout

# ---------------------------------------

tf_version = int(tf.__version__.split(".", maxsplit=1)[0])

if tf_version == 1:
    from keras.models import Model, Sequential
    from keras.layers import (
        Convolution2D,
        ZeroPadding2D,
        MaxPooling2D,
        Flatten,
        Dropout,
        Activation,
        Lambda,
    )
    from keras import backend as K
else:
    from tensorflow.keras.models import Model, Sequential
    from tensorflow.keras.layers import (
        Convolution2D,
        ZeroPadding2D,
        MaxPooling2D,
        Flatten,
        Dropout,
        Activation,
        Lambda,
    )
    from tensorflow.keras import backend as K

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 606.4/606.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 12.1 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116934 sha256=acbabf50e66dccc2ecd3c7d3b730643be45a270f059871723563eccc10102fd1
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire


Resize and segment

In [ ]:
!git clone https://github.com/WillBrennan/SemanticSegmentation.git

Cloning into 'SemanticSegmentation'...
remote: Enumerating objects: 88, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 88 (delta 18), reused 15 (delta 13), pack-reused 49
Receiving objects: 100% (88/88), 2.77 MiB | 24.01 MiB/s, done.
Resolving deltas: 100% (28/28), done.
Filtering content: 100% (4/4), 640.60 MiB | 40.08 MiB/s, done.


In [ ]:
!pip install pytorch-ignite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.4/272.4 kB 4.7 MB/s eta 0:00:00


In [ ]:
from PIL import Image
import os
import shutil
from concurrent.futures import ThreadPoolExecutor

def resize_image(img_path, output_path, size=(224, 224)):
    img = Image.open(img_path)
    img = img.resize(size, Image.LANCZOS)
    img.save(output_path)

def resize_images_in_folder(folder, output_folder, size=(224, 224), max_workers=5):
    # Create the output folder if it does not exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Resize images in the folder using ThreadPoolExecutor
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        for filename in os.listdir(folder):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(folder, filename)
                output_path = os.path.join(output_folder, filename)
                executor.submit(resize_image, img_path, output_path, size)

In [ ]:
resize_images_in_folder('/content/cropped_faces_private_test', '/content/cropped_faces_private_test_224x224')

In [ ]:
!gdown https://drive.google.com/uc?id=<dataset_id>&export=download


Downloading...
From: https://drive.google.com/uc?id=<dataset_id>
To: /content/evaluate_images.py
100% 4.72k/4.72k [00:00<00:00, 16.0MB/s]


In [ ]:
# Write segmented images here

PATH = '/content/SemanticSegmentation/segmented_private_test'
if not os.path.exists(PATH):
        os.makedirs(PATH)
%cd {PATH}
# Please remember to  --save-mask-only
!python /content/SemanticSegmentation/evaluate_images.py \
--images /content/cropped_faces_private_test_224x224 \
--model /content/SemanticSegmentation/pretrained/model_segmentation_skin_30.pth \
--model-type FCNResNet101 \
--output-folder /content/cropped_faces_private_test_224x224_skin_segmentation \
--save \
--save-mask-only


In [ ]:
# ZIP file if you want to use later
folder_path = "/content/cropped_faces_private_test_224x224_skin_segmentation"
zip_name = "cropped_faces_private_test_224x224_skin_segmentation.zip"
!zip -r {zip_name} {folder_path}

!cp "/content/SemanticSegmentation/segmented_private_test/cropped_faces_private_test_224x224_skin_segmentation.zip" "/content/gdrive/MyDrive/Skintone_Classification"

In [ ]:
def preprocess_image_facenet(image_path):
    image_size = (160, 160)
    image = cv2.imread(image_path)

    # Check if the image is loaded properly
    if image is None:
        print(f"Failed to load image: {image_path}")
        return None

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, image_size)
    image = image / 255.0
    image = np.expand_dims(image, axis=0)
    return image.astype(np.float32)

def create_feature_vector(image):
    # Assuming grayscale images, calculate histogram for the single channel, excluding zeros
    hist, _ = np.histogram(image[image != 0], bins=256, range=(1, 255))

    # Normalize the histogram into a single feature vector
    sum_of_vector = hist.sum()

    if sum_of_vector != 0:
        feature_vector = hist / sum_of_vector
    else:
        feature_vector = np.zeros_like(hist)

    # Convert feature vector to float32 and reshape to 2D
    feature_vector = feature_vector.astype(np.float32).reshape(1, -1)

    return feature_vector



def predict_skintone(image_path):
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to load image: {image_path}")
        return "Unknown"

    # Convert to grayscale
    grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    print(f"Grayscale Image Shape: {grayscale_image.shape}")

    # Create feature vector from the grayscale image
    feature_vector_input = create_feature_vector(grayscale_image)
    print(f"Feature vector shape: {feature_vector_input.shape}, Sample: {feature_vector_input[0][:5]}")

    # Predict skintone using the model
    skintone_output = skintone_session.run(None, {'dense_input': feature_vector_input})
    print(f"Raw model output: {skintone_output}")


    return skintone_output

In [ ]:
!gdown https://drive.google.com/uc?id=<dataset_id>&export=download
!gdown https://drive.google.com/uc?id=<dataset_id>&export=download


Downloading...
From: https://drive.google.com/uc?id=<dataset_id>
To: /content/SemanticSegmentation/segmented_private_test/model_race_ptFaceNet_lr0.001_bs64_cw_oversampling_augmented_largerDataset.onnx
100% 96.1M/96.1M [00:00<00:00, 170MB/s]


In [ ]:
import os
import cv2
import onnxruntime
import numpy as np
import pandas as pd
folder_path = '/content/cropped_faces_private_test'
answer_path = '/content/answer_private_test.xlsx'
filename_prefix = 'mask_skin_'

# Load DataFrame from Excel
df = pd.read_excel(answer_path)

prediction_mapping = {0: 'Caucasian', 1: 'Mongoloid', 2: 'Negroid'}
skintone_mapping = {0: 'dark', 1: 'light', 2: 'mid-dark', 3: 'mid-light' }

# Load the skintone prediction ONNX model
skintone_model_path = '/content/model_skintone_histogram_preprocess_image_IQR25_75th.onnx'
skintone_session = onnxruntime.InferenceSession(skintone_model_path)

# Load the facenet model
facenet_session = onnxruntime.InferenceSession('/content/model_race_ptFaceNet_lr0.001_bs64_cw_oversampling_augmented_largerDataset.onnx')
completed_images=0
total_images = len(df)

for index, row in df.iterrows():
    image_file = row['image_file']
    image_path = os.path.join(folder_path, image_file)
    print(image_path)

    # Preprocess the image for facenet
    input_data_facenet = preprocess_image_facenet(image_path)

    # Run inference with facenet
    output_facenet = facenet_session.run(None, {'input_5': input_data_facenet})[0]


    # Combine predictions based on weights
    prediction_race = np.argmax(output_facenet, axis=1)[0]
    prediction_race_label = prediction_mapping.get(prediction_race, 'Unknown')

    # Update DataFrame with prediction
    df.at[index, 'race'] = prediction_race_label


    # SKINTONE
    image_file = filename_prefix + row['image_file']  # Add the prefix here
    image_path = os.path.join('/content/cropped_faces_private_test_224x224_skin_segmentation', image_file)

    output_skintone = predict_skintone(image_path)

    if prediction_race_label == 'Negroid':
              # Create a new list with modified values
              output_skintone[0][0][0]*=0.4739
              output_skintone[0][0][1]*=0.0162
              output_skintone[0][0][2]*=0.5785
              output_skintone[0][0][3]*=0.1693


    predicted_skintone = np.argmax(output_skintone[0])
    print(predicted_skintone)
    predicted_skintone_label = skintone_mapping.get(predicted_skintone, 'Unknown')

    df.at[index, 'skintone'] = predicted_skintone_label


    completed_images += 1
    progress_percentage = (completed_images / total_images) * 100

    print(f"Predicted race for {image_file}: {prediction_race_label}")
    print(f"Progress: {progress_percentage:.2f}%")

df.to_excel(answer_path, index=False, engine='openpyxl')
print("Process completed. Check the updated Excel file.")



